## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.
2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).
3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.
Następnie znaleźć minimum.
4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.
6. Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np

if not os.path.exists("jet.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/jet.bmp --no-check-certificate

jet = cv2.imread('jet.bmp',cv2.IMREAD_GRAYSCALE)
hist = cv2.calcHist([jet],[0],None,[256],[0,256])

c_sum = hist.cumsum()* hist.max()/hist.max()
c_sum = c_sum 


In [ ]:
#wyznaczenie progu przedziału
lm = np.min(np.abs(c_sum - (jet.shape[0]/2*hist.max()/hist.max())))

#rozdzielenie histogramu głównego na dwa histogramy
hist21=hist[0:int(lm)]
hist22=hist[int(lm):jet.shape[0]]

#normalizacja histogramów skumulowanych
cs1=hist21.cumsum()/np.max(hist21)
cs2=hist22.cumsum()/np.max(hist22)

#stworzenie przekształcenia LUT
cs1= lm*cs1
cs2= lm+1+(255-lm-1)*cs2

#połączenie dwóch LUT w jeden (rozciagnięcie, aby wszystie wartości były uwzględniane)
cs1_norm=cv2.normalize(cs1,None,0,lm,cv2.NORM_MINMAX)

cs2_norm=cv2.normalize(cs2,None,lm,255,cv2.NORM_MINMAX)
C=np.append(cs1_norm,cs2_norm)

#przekształcenie
transfor = cv2.LUT(jet,C)
transforH = cv2.calcHist(transfor,[0],None,[256],[0,256])



fig, axs = plt.subplots(1, 2)

fig.set_size_inches(10, 5)
axs[0].imshow(jet, 'gray', vmin=0, vmax=256)
axs[0].axis('off')
axs[1].plot(hist)
axs[1].grid()
axs[1].set_title("histogram orginalny")
axs[0].set_title("obraz orginalny")
plt.show()


fig, axs = plt.subplots(1, 2)

fig.set_size_inches(10, 5)
axs[0].imshow(transfor, 'gray', vmin=0, vmax=256)
axs[0].axis('off')
axs[1].plot(transforH)
axs[1].grid()
axs[1].set_title("histogram DSIHE")
axs[0].set_title("obraz DSIHE")
plt.show()



jetHE = cv2.equalizeHist(jet)
jetHEH = cv2.calcHist(jetHE,[0],None,[256],[0,256])

fig, axs = plt.subplots(1, 2)

fig.set_size_inches(10, 5)
axs[0].imshow(jetHE, 'gray', vmin=0, vmax=256)
axs[0].axis('off')
axs[1].plot(jetHEH)
axs[1].grid()
axs[1].set_title("histogram HE")
axs[0].set_title("obraz HE")
plt.show()




In [ ]:
#obraz po przekształceniu DSIHE ma mniejszy, bardziej naturalny kontrast od obrazu po przekształćeniu HE. Z histogrmów można odczytać
#że obraz DSIHE ma więcej pikseli o wartośći około 85 (histogram bardziej rozciągnięty w górę)